In [1]:
#default_exp utils

# Basic Utils

> Basic utility functions across sac_stac.

In [2]:
#%load_ext autoreload
#%autoreload 2

In [3]:
# from nbdev.showdoc import *

In [4]:
import pandas as pd

In [1]:
import os
import zipfile
import time
from glob import glob
from urllib.parse import urlparse
import gc
import shutil

import boto3
import gdal
from pystac import StacIO
import rasterio
from shapely.geometry import Polygon

from sedas_pyapi.sedas_api import SeDASAPI
from sedas_pyapi.bulk_download import SeDASBulkDownload

## Sedas

Probably best if we start by logging into sedas to be able to access datasets in their current format.

In [8]:
#export
def sedas_client():
    "Log into sedas api."
    sedas = SeDASAPI(os.getenv('SEDAS_USERNAME'), os.getenv('SEDAS_PWD'))
    sedas.base_url="https://geobrowsertest.satapps.org/api/"
    return sedas

In [10]:
sedas_client()

HTTP Error 405: Not Allowed
<html>
<head><title>405 Not Allowed</title></head>
<body>
<center><h1>405 Not Allowed</h1></center>
<hr><center>nginx/1.16.0</center>
</body>
</html>



HTTPError: HTTP Error 405: Not Allowed

should inc. test for env vars...?

And we'll want to find datasets from different *Collections*, which sedas expects as *groups*. Will eventually **need to add test endpoint** to be able to access NovaSAR datasets.

In [ ]:
#export
def sedas_get_collections():
    sedas = sedas_client()
    result_groups = sedas.list_sensor_groups()
    groups = []
    for i in range(0, len(result_groups)):
        groups.append(result_groups[i]['name'])
    return f"Available groups are: {', '.join(groups)}"

In [ ]:
sedas_get_collections()

In [ ]:
#export
def sedas_find_datasets(wkt, startDate, endDate, collection):
    optical_groups = ['S2','Pleiades','SPOT']
    sar_groups = ['S1','Cosmo-SkyMed','AIRSAR']
    sedas = sedas_client()
    if collection in optical_groups: 
        res = sedas.search_optical(wkt, startDate, endDate, source_group=collection) 
    elif collection in sar_groups:
        res = sedas.search_sar(wkt, startDate, endDate, source_group=collection)
    return res

For example for a small area over Oxford we can find...

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products']).head(2)

We can do a single download.

In [ ]:
#export
def sedas_download(sedas_res_dicts, down_dir, sedas=None):
    "Use product dicts list to download into a dir."
    if not sedas:
        sedas = sedas_client()
    downloader = SeDASBulkDownload(sedas, down_dir, parallel=2)
    downloader.add(sedas_res_dicts)
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Downloading")
    while not downloader.is_done():
        time.sleep(5)
    downloader.shutdown()
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Downloaded")

In [ ]:
# tbd example

In [ ]:
#export
def sedas_extract(down_zip, scene_dir):
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Extracting {down_zip}")
    with zipfile.ZipFile(down_zip, 'r') as zip_ref:
        zip_ref.extractall(scene_dir)
    if os.path.exists(scene_dir):
        os.remove(down_zip)
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Extracted to {scene_dir}")

In [ ]:
# tbd example

## Cloud-Optimised Formatting

Once we have access to the datasets, we will want to convert raster images of any format into a default COG format. (It helps that COGs are now an official [gdal driver](https://gdal.org/drivers/raster/cog.html).

In [8]:
#export
def convert2cog(img_path, cog_path, band):
    """
    Convert gdal raster into COG with default settings.
    Considering lzw compression.
    See https://www.cogeo.org/developers-guide.html.
    """
    # translate into new cog file
    kwargs = {
        'format': 'COG',
#         'creationOptions' : ['COMPRESS=LZW'],
        'bandList': [band]
    }
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Starting conversion: {img_path}.")
    ds = gdal.Translate(cog_path, img_path, **kwargs)
    ds = None
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Conversion complete: {os.path.exists(cog_path)}.")

In [9]:
#export
def mosaic2cog(imgs, out_cog, band):
    """
    Mosaic imgs into cog.
    Usual vrt assumptions.
    """
    tmp_vrt = f"{out_cog[:-4]}_mosaic.tif"
    # mosaic into vrt
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Mosaicing band {band} imgs: {imgs}")
    ds = gdal.BuildVRT(tmp_vrt, imgs, bandList=[band])
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Mosaicd {tmp_vrt}")
    # write vrt mosaic to cog
    convert2cog(ds, out_cog, 1)
    ds = None

    if os.path.exists(tmp_vrt):
        os.remove(tmp_vrt)

mosaic currently specific to pleiades ([:-13] for slicing name). Needs to be generalised a little more

In [10]:
#export
def cogmosaicbands(img_paths, numbands, basename):
    """
    Convert bands from img_paths to cog, output 
    basename_band#.tif. Mosaic if >1 img paths.
    """
    if len(img_paths) == 1:
        for b in range(1, numbands+1):
            convert2cog(img_paths[0], f"{basename}_band{b}.tif", b)
        for f in glob(f"{img_paths[0][:-4]}*"):
            os.remove(f)
    elif len(img_paths) > 1:
        for b in range(1, numbands+1):
            mosaic2cog(img_paths[0], f"{basename}_band{b}.tif", b)
        for img_path in img_paths:
            for f in glob(f"{img_path[:-4]}*"):
                os.remove(f)

## Object Storage

In [ ]:
# export
def s3_create_client(s3_bucket):
    """
    Create and set up a connection to S3
    :param s3_bucket:
    :return: the s3 client object.
    """
    access = os.getenv("AWS_ACCESS_KEY_ID")
    secret = os.getenv("AWS_SECRET_ACCESS_KEY")

    session = boto3.Session(
        access,
        secret,
    )
    endpoint_url = os.getenv("AWS_S3_ENDPOINT_URL")

    if endpoint_url is not None:
        s3 = session.resource('s3', endpoint_url=endpoint_url)
    else:
        s3 = session.resource('s3', region_name='eu-west-2')

    bucket = s3.Bucket(s3_bucket)

    if endpoint_url is not None:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret,
            endpoint_url=endpoint_url
        )
    else:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret
        )
    return s3_client, bucket

We can use this to connect to any S3-like object storage end point. Assumes env vars.

In [11]:
#export
gb = 1024 ** 3

In [12]:
#export
def s3_single_upload(in_path, s3_path, s3_bucket):
    """
    put a file into S3 from the local file system.
    :param in_path: a path to a file on the local file system
    :param s3_path: where in S3 to put the file.
    :return: None
    """
    # prep session & creds
    s3_client, bucket = s3_create_client(s3_bucket)

    # Ensure that multipart uploads only happen if the size of a transfer is larger than
    # S3's size limit for non multipart uploads, which is 5 GB. we copy using multipart 
    # at anything over 4gb
    transfer_config = boto3.s3.transfer.TransferConfig(multipart_threshold=2 * gb,
                                                       max_concurrency=10,
                                                       multipart_chunksize=2 * gb,
                                                       use_threads=True)
    s3_client.upload_file(in_path, bucket.name, s3_path)

In [13]:
#export
def get_rel_dir_s3_paths(local_dir, s3_dir):
    """
    returns local path, remote path pair list.
    """  
    paths = []
    for subdir, dirs, files in os.walk(local_dir):
        for file in files:
            full_path = os.path.join(subdir, file)
            paths.append([ full_path, s3_dir + local_dir.split('/')[-2] + '/' + full_path[len(local_dir):] ])
    return paths

In [14]:
#export
def s3_upload_dir(in_dir, s3_bucket, s3_dir):
    """
    Upload all items in directory, inc. dir.
    """
    paths = get_rel_dir_s3_paths(in_dir, s3_dir)
    upload_list = [(in_path, out_path, s3_bucket)
                   for in_path, out_path in paths]
    for i in upload_list:
        s3_single_upload(i[0], i[1], i[2])

In [15]:
# export
def s3_list_objects_paths(s3_bucket, prefix):
    """List of paths only returned, not full object responses"""
    client, bucket = s3_create_client(s3_bucket)
    
    return [e['Key'] for p in client.get_paginator("list_objects_v2").paginate(Bucket=s3_bucket, Prefix=prefix) for e in p['Contents']]


In [17]:
#export
def s3_list_objects(s3_bucket, prefix):
    # prep session & creds
    client, bucket = s3_create_client(s3_bucket)
    response = client.list_objects_v2(Bucket=s3_bucket, Prefix=prefix)

    return response

## General

In [ ]:
#export
def clean_up(work_dir):
    # TODO: sort out logging changes...
    gc.collect()
    shutil.rmtree(work_dir)
    pass

In [ ]:
#export
def setup_logging():

    logging.basicConfig(level=logging.DEBUG)
    root = logging.getLogger()
    root.setLevel(os.environ.get("LOGLEVEL", "DEBUG"))

    # Turn down rasterio. It is extremely chatty at debug level.
    logging.getLogger("rasterio").setLevel("INFO")
    logging.getLogger("rasterio._io").setLevel("WARNING")

    # Boto Core is also very chatty at debug. Logging entire request text etc
    logging.getLogger("botocore").setLevel("INFO")
    logging.getLogger("boto").setLevel("INFO")
    logging.getLogger("boto3.resources").setLevel("INFO")
    logging.getLogger("s3transfer").setLevel("INFO")
    logging.getLogger("urllib3").setLevel("INFO")

    return root

In [ ]:
#export
def get_img_crs_and_bbox(raster_uri):
    """
    BBOX list, geometry shapely and rasterio crs from
    URI of COG.
    nb: footprint currently same as bbo.
    """
    with rasterio.open(raster_uri) as ds:
        bounds = ds.bounds
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
        footprint = Polygon([
            [bounds.left, bounds.bottom],
            [bounds.left, bounds.top],
            [bounds.right, bounds.top],
            [bounds.right, bounds.bottom]
        ])
        return bbox, footprint, ds.crs

## STAC

We can amend the default STAC I/O tools to work with our object storage.

First up is to be able to create a uri given an endpoint, bucket name and object path.

In [ ]:
#export
def create_uri(s3_path, s3_bucket='public-eo-data', endpoint_url=os.getenv('AWS_S3_ENDPOINT_URL')):    
    return f"{endpoint_url}/{s3_bucket}/{s3_path}"

Amend the reading method.

In [ ]:
#export
def my_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme == 's3':
        bucket = parsed.netloc
        key = parsed.path[1:]
        s3 = boto3.resource('s3')
        obj = s3.Object(bucket, key)
        return obj.get()['Body'].read().decode('utf-8')
    else:
        return StacIO.default_read_text_method(uri)

And the writing method.

In [ ]:
#export
def my_write_method(uri, txt):
    parsed = urlparse(uri)
#     print(parsed)
    if (parsed.scheme == 'https') and ('s3' in parsed.netloc):
        endpoint_url = os.getenv("AWS_S3_ENDPOINT_URL")
        s3_bucket = parsed.path.split('/')[1]
        
        s3_client, bucket = s3_create_client(s3_bucket)
        
        key = '/'.join(parsed.path.split('/')[2:])
#         print(key)
        s3_client.put_object(Body=txt, Bucket=s3_bucket, Key=key)
        
    else:
        print('relaying to defaul')
        StacIO.default_write_text_method(uri, txt)

Then allow us to overwrite the defaults.

In [4]:
#export
def pystac_setIO():
    StacIO.read_text_method = my_read_method
    StacIO.write_text_method = my_write_method

## Export

In [ ]:
# from nbdev.export import notebook2script; notebook2script()